In [ ]:
!wget http://cb.lk/covid_19

--2020-11-06 16:16:11--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 159.89.170.74
Connecting to cb.lk (cb.lk)|159.89.170.74|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cb.lk/covid_19 [following]
--2020-11-06 16:16:12--  https://cb.lk/covid_19
Connecting to cb.lk (cb.lk)|159.89.170.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2020-11-06 16:16:13--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2020-11-06 16:16:13--  https://www.dropbox.com/s/raw/7rjw

In [ ]:
!unzip covid_19

Archive:  covid_19
replace CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg  
  inflating: CovidDataset/Val/Covid/85E52EB3-56E9-4D67-82DA-DEA247C82886.jpeg  
  inflating: CovidDataset/Val/Covid/4-x-day1.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-53.jpg  
  inflating: CovidDataset/Val/Covid/4e43e48d52c9e2d4c6c1fb9bc1544f_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-35-1.jpg  
  inflating: CovidDataset/Val/Covid/53EC07C9-5CC6-4BE4-9B6F-D7B0D72AAA7E.jpeg  
  inflating: CovidDataset/Val/Covid/6CB4EFC6-68FA-4CD5-940C-BEFA8DAFE9A7.jpeg  
  inflating: CovidDataset/Val/Covid/58cb9263f16e94305c730685358e4e_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/6b3bdbc31f65230b8cdcc3cef5f8ba8a-40ac-0.jpg  
  inflating: CovidDataset/Val/Covid/auntminnie-d-2020_01_

In [ ]:
yTRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Val"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [ ]:
train_data = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
test_dataset = image.ImageDataGenerator(
    rescale = 1./255
)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras import layers

# base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
# include_top = False, # Leave out the last fully connected layer
# weights = 'imagenet')


# for layer in base_model.layers:
#     layer.trainable = False

# # Flatten the output layer to 1 dimension
# x = layers.Flatten()(base_model.output)

# # Add a fully connected layer with 512 hidden units and ReLU activation
# x = layers.Dense(512, activation='relu')(x)

# # Add a dropout rate of 0.5
# x = layers.Dropout(0.5)(x)

# # Add a final sigmoid layer for classification
# x = layers.Dense(1, activation='sigmoid')(x)

# model = keras.models.Model(base_model.input, x)
# adam = Adam(lr=0.0001)

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_70 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_70 (Dropout)         (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_71 (Dropout)         (None, 54, 54, 64)      

In [ ]:
train_generator = train_data.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 224 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [ ]:
validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary',
)

Found 60 images belonging to 2 classes.


In [ ]:
from tensorflow.keras import backend as K
lr = 0.001


class printlearningrate(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        optimizer = self.model.optimizer
        lr = K.eval(optimizer.lr)
        Epoch_count = epoch + 1
        print('\n', "Epoch:", Epoch_count, ', LR: {:.3f}'.format(lr))

printlr = printlearningrate() 

def scheduler(epoch):
  optimizer = model.optimizer
  return K.eval(optimizer.lr + 0.005)

updatelr = keras.callbacks.LearningRateScheduler(scheduler)




In [ ]:

# hist = model.fit_generator(
#     train_generator,
#     steps_per_epoch=6,
#     epochs = 50,
#     validation_data=validation_generator,
#     validation_steps=2,
#     callbacks = [printlr,updatelr]
# )
VAL_ACCURACY_THRESHOLD = 0.95
ACCURACY_THRESHOLD = 0.91
class myCallback(keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > ACCURACY_THRESHOLD and logs.get('val_accuracy') > ACCURACY_THRESHOLD):   
          print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
          self.model.stop_training = True

callbacks = myCallback()

hist = model.fit(train_generator,
                 steps_per_epoch = 6, 
                 epochs = 50,
                 validation_data = validation_generator, 
                 validation_steps=2
                 )

Epoch 1/50
6/6 [==============================] - 9s 2s/step - loss: 0.6054 - accuracy: 0.6510 - val_loss: 0.5128 - val_accuracy: 0.8167
Epoch 2/50
6/6 [==============================] - 7s 1s/step - loss: 0.6091 - accuracy: 0.8073 - val_loss: 0.5225 - val_accuracy: 0.9333
Epoch 3/50
6/6 [==============================] - 8s 1s/step - loss: 0.5533 - accuracy: 0.9219 - val_loss: 0.4331 - val_accuracy: 0.9333
Epoch 4/50
6/6 [==============================] - 8s 1s/step - loss: 0.5003 - accuracy: 0.8281 - val_loss: 0.4042 - val_accuracy: 0.8333
Epoch 5/50
6/6 [==============================] - 8s 1s/step - loss: 0.5634 - accuracy: 0.8229 - val_loss: 0.4350 - val_accuracy: 0.7667
Epoch 6/50
6/6 [==============================] - 8s 1s/step - loss: 0.4397 - accuracy: 0.8646 - val_loss: 0.4243 - val_accuracy: 0.8500
Epoch 7/50
6/6 [==============================] - 8s 1s/step - loss: 0.5414 - accuracy: 0.7917 - val_loss: 0.4522 - val_accuracy: 0.8667
Epoch 8/50
6/6 [=========================

In [ ]:
model.save('final_model.model')

INFO:tensorflow:Assets written to: final_model.model/assets


In [ ]:
!pip install skimage
import cv2
import tensorflow as tf
import urllib
import numpy as np
from skimage.color import gray2rgb
from skimage import io
import requests
CATEGORIES = ["covid positive", "not covid positive"]

from PIL import Image

def prepare(filepath):
  IMG_SIZE = 224
  resp = urllib.request.urlopen(filepath)
  respl = resp.read()
  arr = np.asarray(bytearray(respl), dtype=np.uint8)
  new_array = cv2.resize(arr, (IMG_SIZE, IMG_SIZE))
  stacked_img = np.stack((new_array,)*3, axis=-1)
  print(stacked_img.shape)
  if len(new_array.shape) > 2 and new_array.shape[2] == 4:
    new_array = cv2.cvtColor(new_array, cv2.COLOR_BGRA2BGR)

  return stacked_img.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
new_model = tf.keras.models.load_model("final_model.model")

prediction = new_model.predict([prepare('c19.jpg')])
print(CATEGORIES[int(prediction[0][0])])

In [ ]:
import cv2
import tensorflow as tf
import urllib
import numpy as np
from skimage.color import gray2rgb
from skimage import io
import requests
from io import StringIO, BytesIO
from imread import imread_from_blob
CATEGORIES = ["covid positive", "not covid positive"]

from PIL import Image

def prepare2(filepath):
  IMG_SIZE = 224
  image = io.imread(filepath)
  image = np.asarray(image, dtype="uint8")
  new_array = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
  
  print(new_array.shape)
  if len(new_array.shape) > 2 and image.shape[2] == 4:
    new_array = cv2.cvtColor(new_array, cv2.COLOR_BGRA2RGB)
  img = Image.fromarray(new_array, 'RGB')
  img.save("testfile.png")
  img.show()
  return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
new_model = tf.keras.models.load_model("final_model.model")

prediction = new_model.predict([prepare2('https://lh3.googleusercontent.com/proxy/KR_kqttQM4ECbPE-4RmyWdSHADTVSIEqBdkytcNnvoQbQhQP6HxoRST0P3QtDlRZ5Ja2kd9TNv0gllyIYB3rWuOvibJALIXn4I8_k5f4MaVrp1bKFIxznTm1xFENYIo580DRaw')])
print(CATEGORIES[int(prediction[0][0])])

(224, 224, 3)
covid positive


In [ ]:
!zip -r /content/final_model.model.zip /content/final_model.model/

  adding: content/final_model.model/ (stored 0%)
  adding: content/final_model.model/saved_model.pb (deflated 89%)
  adding: content/final_model.model/assets/ (stored 0%)
  adding: content/final_model.model/variables/ (stored 0%)
  adding: content/final_model.model/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: content/final_model.model/variables/variables.index (deflated 68%)
